In [1]:
from scipy import signal as sig

# imports from custom library
import sys
sys.path.append('../../')
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
from mlrefined_libraries import convnets_library as convlib
from mlrefined_libraries import basics_library as baslib
from mlrefined_libraries import superlearn_library as superlearn
from mlrefined_libraries import multilayer_perceptron_library as network_lib
from autograd import grad as compute_grad  

import autograd.numpy as np
from autograd import grad as compute_grad   
import numpy as npo

import pandas as pd
import cv2
import csv
import pickle
import glob
import time
import copy
from datetime import datetime 

#this is needed to compensate for matplotlib notebook's tendancy to blow up images when plotted inline
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

%load_ext autoreload
%autoreload 2

## Load in data

Load in data.

In [2]:
# load in dataset
datapath = '../../mlrefined_datasets/convnet_datasets/feat_face_data.csv'
data = np.loadtxt(datapath,delimiter = ',')

Contrast normalize images.

In [3]:
# extract input tensor
tensor = data[:,:-1]
y = data[:,-1:]

# contrast normalize image data
def contrast_normalize(data):
    data_means = np.mean(data,axis = 0)
    data = data - data_means
    data_stds = np.std(data,axis = 0)
    data = data/data_stds
    return data

# contrast normalize the input 
tensor_decontrast = contrast_normalize(tensor.T).T

# make new data based on fixed kernel convolutions
new_data = np.concatenate((tensor_decontrast,y),axis = 1)

## learned convolution features

In [88]:
# load in data - for this one split up training and testing
ind = np.random.permutation(len(new_data))

# split it up
data_train = new_data[ind[:3],:]
data_test = new_data[ind[3:],:]

# load data into network
demo = convlib.network_learner.Network()
demo.input_data(data_train,data_test,normalize = True)

In [89]:
# choose cost function
demo.choose_cost(cost_name = 'twoclass_softmax')

# setup network architecture
activation_name = 'linear'
num_kernels = 1
layer_sizes = [64,2,1]
demo.architecture_settings(activation_name,layer_sizes,num_kernels)

In [90]:
# setup optimizer
demo.optimizer_settings(alpha = 10**(-1),max_its = 10,version = 'unnormalized',scale = 0.1)

In [91]:
# demo.fit(verbose = True)

In [92]:
# # compute cost plots on training and testing data
# demo.compute_cost_plots()

# # produce cost functio plots for training and testing data
# demo.plot_histories(start = 0)

The prediction function.

In [93]:
# demo.weight_history[6]
# demo.w_init

# just the gradient

The gradient

In [94]:
test_grad = compute_grad(demo.training_cost)
wa = test_grad(demo.w_init)

(3, 65)
(65, 2)
(1, 65, 2)
Autograd ArrayBox with value [[-0.12022168 -0.13153025]
 [-0.03854317 -0.18003865]
 [-0.12064693 -0.33380425]]


In [83]:
demo.w_init[1]

array([[[-0.08748114, -0.05287551, -0.12917575],
        [-0.02690774,  0.05879566, -0.03019546],
        [ 0.00932586,  0.16003246,  0.02650719]]])

In [76]:
wa

[[array([[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.

In [68]:
demo.w_init[1]

array([[[ 0.09398295,  0.06761082,  0.0184683 ],
        [-0.19839479,  0.12737906, -0.04069006],
        [ 0.03058614, -0.02851461, -0.10171031]]])

In [15]:
np.shape(demo.x_train)

(10, 784)